In [19]:
from PIL import Image
import numpy as np
import pandas as pd
import xarray as xr
import platform
from pathlib import Path
import tensorflow as tf
import math
from tensorflow import keras,math
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

    
def read_data(csv_file_path='./thermal_barrierlife_prediction/data/train-orig.csv', tiff_folder_path='./thermal_barrierlife_prediction//data/train/'):
    df_meta = pd.read_csv(csv_file_path)
    im_list = []
    for i,imid in enumerate(df_meta.Image_ID):
        im = Image.open(tiff_folder_path+str(imid)+'.tif')
        if np.array(im).shape[1]>1024:
            im_list.append(np.array(im))
        if np.array(im).shape[1]<=1024:
            im = im.resize((2048,2048))
            im_list.append(np.array(im))

    ds = xr.Dataset({'greyscale':xr.DataArray(np.array(im_list))})
    ds = ds.rename({'dim_0':'image_id', 'dim_1':'x', 'dim_2':'y'})
    ds['sample'] = 'image_id', df_meta.Sample
    ds['lifetime'] = 'image_id', df_meta.Lifetime
    ds['magnification'] = 'image_id', df_meta.Magnification
    ds['uncertainty'] = 'image_id', df_meta.Uncertainty
    ds['image_id'] = 'image_id', df_meta.Image_ID
    return ds



In [3]:
data_frame = read_data()
X = data_frame['greyscale']
Y = data_frame['lifetime']

image = np.expand_dims(X[15,:,:],-1)

In [4]:
#Functions to display the images used in this experiment
def show_image_batch(images: list):
    """
    Displays a batch of image present in images
    """
    fig = plt.figure(figsize=(50,50))
    for idx in range(10):
        ax = plt.subplot(5, 2, idx+1)
        plt.imshow(images[idx])
        plt.axis("off")

def show_dataset(dataset):
    batch = next(iter(dataset))
    images, labels = batch
    
    plt.figure(figsize=(10, 10))
    for idx in range(9):
        ax = plt.subplot(3, 3, idx + 1)
        plt.imshow(images[idx].numpy().astype("uint8"))
        plt.title("Class: {}".format(class_names[labels[idx].numpy()]))
        plt.axis("off")

      """Preprocess each coordinate — scaled from [0, 1) — by converting each
      coordinate to a random fourier feature, as determined by a matrix with values
      samples from a Gaussian distribution.
      """

In [22]:
import tensorflow as tf
import numpy as np


# ============================== FOURIER MAPPING ===============================
def initialize_fourier_mapping_vector(m, sigma):
    d = 2
    B = tf.random.normal((m, d)) * sigma
    return B
    
def fourier_mapping(coords, B):
    sin_features = tf.math.sin((2 * np.pi) * (tf.matmul(coords, B, transpose_b=True)))
    cos_features = tf.math.cos((2 * np.pi) * (tf.matmul(coords, B, transpose_b=True)))
    features = tf.concat([sin_features, cos_features], axis=-1)
    return features


The folloiwng function: Generally used as the input to a CPPN, but can also augment each layer of a ConvNet with location hints
  """Return a 2x2 matrix where the values at each location are equal to the
  indices of that location
  """

In [10]:
# ============================== DATA ==========================================
def get_coord_ints(y_dim, x_dim):
    ys = tf.range(y_dim)[tf.newaxis]
    xs = tf.range(x_dim)[:, tf.newaxis]
    coord_ints = tf.stack([ys+xs-ys, xs+ys-xs], axis=2)
    return coord_ints

def generate_scaled_coordinate_hints(batch_size, img_dim=2048):
    spatial_scale = 1. / img_dim
    coord_ints = get_coord_ints(img_dim, img_dim)
    coords = tf.cast(coord_ints, tf.float32)
    coords = tf.stack([coords[:, :, 0] * spatial_scale,
                     coords[:, :, 1] * spatial_scale], axis=-1)
    coords = tf.tile(coords[tf.newaxis], [batch_size, 1, 1, 1])
    return coords

    

In [23]:
batch_size = 1#x.shape[0]
coords = generate_scaled_coordinate_hints(batch_size)
mapping_size = 256
sigma = 10
B = initialize_fourier_mapping_vector(m=mapping_size, sigma=sigma)
 #if self.use_fourier_features:
features = fourier_mapping(coords, B=B)

In [24]:
features.shape

TensorShape([1, 2048, 2048, 512])